In [21]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('/Users/krys/Desktop/UNIVERSITY/Year 1/Data Mining and Machine Learning/AirBnb datasets/barcelona/listings.csv')

In [22]:
# Identify columns where all values are the same
constant_columns = [col for col in listings.columns if listings[col].nunique() == 1]

# Print the columns where all values are the same
print("Columns where all values are the same:", constant_columns)

Columns where all values are the same: ['scrape_id', 'last_scraped', 'calendar_last_scraped']


In [23]:
# Remove columns where all values are the same
columns_to_remove = ['scrape_id', 'last_scraped', 'calendar_last_scraped']
listings = listings.drop(columns=columns_to_remove)

In [24]:
# Remove irrelevant or uninformative features
columns_to_remove = ['listing_url', 'source', 'name', 'description', 'neighborhood_overview',
                     'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                     'host_picture_url', 'license', 'calendar_updated']

listings = listings.drop(columns=columns_to_remove)

In [25]:
# Convert boolean values into binary representations
listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f': 0})
listings['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f': 0})
listings['has_availability'] = listings['has_availability'].replace({'t': 1, 'f': 0})

/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_51457/25808645.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_51457/25808645.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_51457/25808645.py:4: FutureWarning: Downcasting beh

In [26]:
# Remove the dollar symbol and commas, then convert the column to numeric
listings['price'] = listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float)


Importing amenities from "amenities.csv" dataset

In [27]:
amenities_dummies = pd.read_csv('amenities.csv')

In [28]:
# Concatenate the original DataFrame (excluding the old 'amenities' column) with the new boolean features
listings = pd.concat([listings.drop(columns=['amenities']), amenities_dummies], axis=1)

In [29]:
listing_ids = listings['id']

print(listing_ids)

listings = listings.drop(columns = ['id'])

0                     269467
1                     270197
2                      18674
3                     272282
4                      23197
                ...         
18920    1173891409488923931
18921    1178741129427938140
18922    1178910876458935142
18923    1173984319228839137
18924    1174000036524843870
Name: id, Length: 18925, dtype: int64


In [30]:
from sklearn.preprocessing import StandardScaler

# Select numeric columns for scaling
numeric_columns = listings.select_dtypes(include=['float64', 'int64']).columns
numeric_data = listings[numeric_columns]

# Scale the numeric data
scaler = StandardScaler()
numeric_data_scaled = scaler.fit_transform(numeric_data)

# Convert the scaled data back to a DataFrame
numeric_data_scaled_df = pd.DataFrame(numeric_data_scaled, columns=numeric_columns)

# Combine scaled numeric data with non-numeric data
non_numeric_data = listings.drop(columns=numeric_columns)
listings = pd.concat([listing_ids, non_numeric_data.reset_index(drop=True), numeric_data_scaled_df], axis=1)

listings

,id,host_since,host_location,host_response_time,host_neighbourhood,host_verifications,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms_text,first_review,last_review,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Avena,Bar,Bidet,Bikes,Blender,Boat_slip,Bowling_alley,Bread_maker,Canal_view,Candy_-_FCS100X/E_-_Horno_el\u00e9ctrico_-_2100W,Climbing_wall,Desert_view,Dishes_and_silverware,Foosball_table,Gated_property,Golf_course_view,Hammock,Hockey_rink,Ikea,Ironing_board,Kayak,Kenwood,Lake_view,Laser_tag,Lavandin,Lockbox,Marca_blanca,Marina_view,Media_room,Mini_golf,Mosquito_net,Mountain_view,Nevera_y_congelador,Noise_decibel_monitors_on_property,Palmolive,Pantene,Piano,Private_sauna,Property_manager,Record_player,Rice_maker,River_view,Samsung,Sauna,Sea_view,Sempre_hi_ha_xamp\u00fa,Shared_sauna,Siemens_Inoxidable_con_Vapor,Skate_ramp,Ski-in/Ski-out,Sonos_(Spotify_Connect,Sony,Sunstech,Table_corner_guards,Theme_room,Toaster,Valley_view,Vineyard_view,Waterfront,1_day_a_week_-_available_at_extra_cost,2_days_a_week_-_available_at_extra_cost,2-5_years_old,4_days_a_week_-_available_at_extra_cost,5_days_a_week_-_available_at_extra_cost,5-10_years_old,AirPlay,Apple,Fructis,Inox,Nivea,PS2,Puerta_desmontable_de_doble_cristal_con_efecto_espejo,Thursday_-_available_at_extra_cost,Tuesday,Wednesday,Unnamed: 75,and_10+_years_old,and_5-10_years_old,and_PS3,and_dresser,and_wardrobe,closet,every_day_-_available_at_extra_cost,every_day_-_included_with_your_stay,free_weights,olympic-sized,peque\u00f1a,rowing,wardrobe,water_slide,wood-burning,wifi_aggregated,shampoo_aggregated,body_soap_aggregated,tv_aggregated,refrigerator_aggregated,oven_aggregated,stove_aggregated,sound_system_aggregated,bbq_aggregated,games_aggregated,baby_aggregated,baking_aggregated,booster_seat_aggregated,ceiling_fan_aggregated,changing_table_aggregated,city_skyline_view_aggregated,clothing_storage_aggregated,coffee_aggregated,crib_aggregated,dryer_aggregated,charger_aggregated,dove_aggregated,elevator_aggregated,essentials_aggregated,ethernet_aggregated,exercise_equipment_aggregated,cameras_aggregated,fire_extinguisher_aggregated,first_kit_aggregated,fire_pit_aggregated,high_chair_aggregated,carport_aggregated,parking_aggregated,washer_aggregated,freezer_aggregated,garden_aggregated,housekeeping_aggregated,keypad_aggregated,kitchen_aggregated,lock_on_bedroom_aggregated,cleaning_aggregated,pool_aggregated,long_term_aggregated,luggage_dropoff_aggregated,microwave_aggregated,outdoor_aggregated,ocean_view_aggregated,gym_aggregated,hot_tub_aggregated,bay_aggregated,garage_aggregated,resort_aggregated,hangers_aggregated,heating_aggregated,host_greets_aggregated,hot_water_aggregated,pets_aggregated,private_entrance_aggregated,balcony_aggregated,safe_aggregated,self_check_in_aggregated,single_level_home_aggregated,smoke_alarm_aggregated,smoking_allowed_aggregated,sun_loungers_aggregated,prime_video_aggregated,chromecast_aggregated,dvd_aggregated,monday_to_friday_aggregated,disney_aggregated,netflix_aggregated,roku_aggregated,saturday_aggregated,sunday_aggregated,bathtub_aggregated,bed_linens_aggregated,books_and_

In [ ]:
sentiment = pd.read_csv('sentiment.csv')

In [ ]:
listings['id'] = listings['id'].astype(str)
sentiment['listing_id'] = sentiment['listing_id'].astype(str)

# Merge the datasets on the corresponding ID columns
merged_data = pd.merge(listings, sentiment, left_on='id', right_on='listing_id', how='left')

# Drop the duplicate 'listing_id' column if it exists
merged_data = merged_data.drop(columns=['listing_id'])

In [33]:
merged_data

,id,host_since,host_location,host_response_time,host_neighbourhood,host_verifications,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms_text,first_review,last_review,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Avena,Bar,Bidet,Bikes,Blender,Boat_slip,Bowling_alley,Bread_maker,Canal_view,Candy_-_FCS100X/E_-_Horno_el\u00e9ctrico_-_2100W,Climbing_wall,Desert_view,Dishes_and_silverware,Foosball_table,Gated_property,Golf_course_view,Hammock,Hockey_rink,Ikea,Ironing_board,Kayak,Kenwood,Lake_view,Laser_tag,Lavandin,Lockbox,Marca_blanca,Marina_view,Media_room,Mini_golf,Mosquito_net,Mountain_view,Nevera_y_congelador,Noise_decibel_monitors_on_property,Palmolive,Pantene,Piano,Private_sauna,Property_manager,Record_player,Rice_maker,River_view,Samsung,Sauna,Sea_view,Sempre_hi_ha_xamp\u00fa,Shared_sauna,Siemens_Inoxidable_con_Vapor,Skate_ramp,Ski-in/Ski-out,Sonos_(Spotify_Connect,Sony,Sunstech,Table_corner_guards,Theme_room,Toaster,Valley_view,Vineyard_view,Waterfront,1_day_a_week_-_available_at_extra_cost,2_days_a_week_-_available_at_extra_cost,2-5_years_old,4_days_a_week_-_available_at_extra_cost,5_days_a_week_-_available_at_extra_cost,5-10_years_old,AirPlay,Apple,Fructis,Inox,Nivea,PS2,Puerta_desmontable_de_doble_cristal_con_efecto_espejo,Thursday_-_available_at_extra_cost,Tuesday,Wednesday,Unnamed: 75,and_10+_years_old,and_5-10_years_old,and_PS3,and_dresser,and_wardrobe,closet,every_day_-_available_at_extra_cost,every_day_-_included_with_your_stay,free_weights,olympic-sized,peque\u00f1a,rowing,wardrobe,water_slide,wood-burning,wifi_aggregated,shampoo_aggregated,body_soap_aggregated,tv_aggregated,refrigerator_aggregated,oven_aggregated,stove_aggregated,sound_system_aggregated,bbq_aggregated,games_aggregated,baby_aggregated,baking_aggregated,booster_seat_aggregated,ceiling_fan_aggregated,changing_table_aggregated,city_skyline_view_aggregated,clothing_storage_aggregated,coffee_aggregated,crib_aggregated,dryer_aggregated,charger_aggregated,dove_aggregated,elevator_aggregated,essentials_aggregated,ethernet_aggregated,exercise_equipment_aggregated,cameras_aggregated,fire_extinguisher_aggregated,first_kit_aggregated,fire_pit_aggregated,high_chair_aggregated,carport_aggregated,parking_aggregated,washer_aggregated,freezer_aggregated,garden_aggregated,housekeeping_aggregated,keypad_aggregated,kitchen_aggregated,lock_on_bedroom_aggregated,cleaning_aggregated,pool_aggregated,long_term_aggregated,luggage_dropoff_aggregated,microwave_aggregated,outdoor_aggregated,ocean_view_aggregated,gym_aggregated,hot_tub_aggregated,bay_aggregated,garage_aggregated,resort_aggregated,hangers_aggregated,heating_aggregated,host_greets_aggregated,hot_water_aggregated,pets_aggregated,private_entrance_aggregated,balcony_aggregated,safe_aggregated,self_check_in_aggregated,single_level_home_aggregated,smoke_alarm_aggregated,smoking_allowed_aggregated,sun_loungers_aggregated,prime_video_aggregated,chromecast_aggregated,dvd_aggregated,monday_to_friday_aggregated,disney_aggregated,netflix_aggregated,roku_aggregated,saturday_aggregated,sunday_aggregated,bathtub_aggregated,bed_linens_aggregated,books_and_